# Set up

In [1]:

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

In [10]:
doc_dict2 = dict()

# Original scripts from `chat_functions.py`

In [ ]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(documents, site_key, vector_dict=vector_dict, text_splitter=None):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    embeddings_dict[site_key] = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)

    vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict


# Iteration 1: `create_retriever_and_description_dicts` function

In [6]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(documents, site_key, vector_dict=vector_dict, text_splitter=None):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    embeddings_dict[site_key] = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)

    vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict


def create_retriever_and_description_dicts(params_dict, doc_dict=doc_dict, vector_dict=vector_dict):
    retriever_dict = dict()
    description_dict = dict()
    for doc_id in doc_dict:
        retriever_dict[doc_id] = create_retriever(
            doc_dict[doc_id], parms_dict[doc_id]['site_key'], 
            vector_dict=vector_dict, 
            text_splitter=parms_dict[doc_id].get('text_splitter', None)
            )
        description_dict[parms_dict[doc_id]['site_key']] = parms_dict['doc_description']

    return retriever_dict, description_dict

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list


recylebc = """
This document provides information from the Recycle BC website or BC government 
website. It has the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""

CoV_mattress = """
Information from the City of Vancouver website about how to recycle mattresses.
"""

params_dict = {
    1: {
        'site_key': 'recycle',
        'doc_description': recylebc,
        'text_splitter': None
    },
    2: {
        'site_key': 'matttress',
        'doc_description': CoV_mattress,
        'text_splitter': None
    }
}

print(params_dict[1].get('text_splitter', 'hello'))

hello


# Iteration 2 [Cache the embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/caching_embeddings#:~:text=Embeddings%20can%20be%20stored%20or%20temporarily%20cached%20to,is%20used%20as%20the%20key%20in%20the%20cache.)

In [13]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(
    documents, site_key, filestore, 
    embeddings_dict=embeddings_dict, 
    vector_dict=vector_dict, text_splitter=None
    ):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)

    try:    
        vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    except KeyError:        
        underlying_embeddings = OpenAIEmbeddings(
            openai_organization=os.environ['openai_organization'],
            openai_api_key=os.environ['openai_api_key']
            )
        embeddings_dict[site_key] = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings, filestore, namespace=underlying_embeddings.model
            )
        vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict


def create_retriever_and_description_dicts(params_dict, filestore, doc_dict=doc_dict, vector_dict=vector_dict):
    retriever_dict = dict()
    description_dict = dict()
    for doc_id in doc_dict:
        retriever_dict[doc_id] = create_retriever(
            doc_dict[doc_id], params_dict[doc_id]['site_key'], 
            filestore,
            vector_dict=vector_dict, 
            text_splitter=params_dict[doc_id].get('text_splitter', None)
            )
        description_dict[parms_dict[doc_id]['site_key']] = parms_dict['doc_description']

    return retriever_dict, description_dict

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list


recylebc = """
This document provides information from the Recycle BC website or BC government 
website. It has the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""

CoV_mattress = """
Information from the City of Vancouver website about how to recycle mattresses.
"""

params_dict = {
    # 1: {
    #     'site_key': 'recycle',
    #     'doc_description': recylebc,
    #     'text_splitter': None
    # },
    2: {
        'site_key': 'mattress',
        'doc_description': CoV_mattress,
        'text_splitter': None
    }
}
filestore = '../embeddings/'

doc_id = 2
try:
    directory = 'data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)
except:
    directory = '../data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)


retriever_dict, description_dict = create_retriever_and_description_dicts(params_dict, filestore)

100%|██████████| 1/1 [00:00<00:00, 246.56it/s]

Number of files: 1


AttributeError: 'str' object has no attribute 'mget'

In [15]:
doc_dict[2]

[Document(page_content="\nWhen you recycle your mattress (foam or futon) and box spring, you reduce the amount of waste going to our landfill.\nTo recycle your mattress and box spring, you can:\n\nBring it to the \xa0transfer station or landfill\xa0for a fee\nHire\xa0a mattress recycling company or junk removal company\n\nImportant note Don't dump your mattress in an alley or on the street.\xa0It's illegal and you could be fined for illegal dumping.\nView the fee, limits, and locations to recycle a\xa0mattress, futon, and box spring\nhttps://vancouver.ca/home-property-development/how-to-dispose-of-mattresses.aspx", metadata={'source': '..\\data\\mattress.txt'})]

# 2.1

In [26]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(
    documents, site_key, filestore, 
    embeddings_dict=embeddings_dict, 
    vector_dict=vector_dict, text_splitter=None
    ):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)
   
    underlying_embeddings = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    embeddings_dict[site_key] = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings, filestore, namespace=underlying_embeddings.model
        )
    # vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    # retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict


def create_retriever_and_description_dicts(params_dict, filestore, doc_dict=doc_dict, vector_dict=vector_dict):
    retriever_dict = dict()
    description_dict = dict()
    for doc_id in doc_dict:
        retriever_dict[doc_id] = create_retriever(
            doc_dict[doc_id], params_dict[doc_id]['site_key'], 
            filestore,
            vector_dict=vector_dict, 
            text_splitter=params_dict[doc_id].get('text_splitter', None)
            )
        description_dict[params_dict[doc_id]['site_key']] = params_dict[doc_id]['doc_description']

    return retriever_dict, description_dict

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list


recylebc = """
This document provides information from the Recycle BC website or BC government 
website. It has the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""

CoV_mattress = """
Information from the City of Vancouver website about how to recycle mattresses.
"""

params_dict = {
    # 1: {
    #     'site_key': 'recycle',
    #     'doc_description': recylebc,
    #     'text_splitter': None
    # },
    2: {
        'site_key': 'mattress',
        'doc_description': CoV_mattress,
        'text_splitter': None
    }
}
filestore = '../embeddings/'

doc_id = 2
try:
    directory = 'data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)
except:
    directory = '../data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)


retriever_dict, description_dict = create_retriever_and_description_dicts(params_dict, filestore)

100%|██████████| 1/1 [00:00<00:00, 901.03it/s]

Number of files: 1


In [19]:
print(params_dict[doc_id].get('text_splitter', None))

None


## 2.2

In [27]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st

directory='../data/' # This is the directory containing the CSV/text files.

# Initialize Dictionaries
tool_dict = dict()
embeddings_dict = dict()
db_dict = dict()
retriever_dict = dict()
vector_dict = dict()
description_dict = dict()
answer_dict=dict()
conversation_dict = dict()
doc_dict = dict()
queries_dict = dict()

def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(
    documents, site_key, filestore, 
    embeddings_dict=embeddings_dict, 
    vector_dict=vector_dict, text_splitter=None
    ):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)
   
    underlying_embeddings = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    embeddings_dict[site_key] = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings, filestore, namespace=underlying_embeddings.model
        )
    # vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    # retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    # return retriever_dict
    return embeddings_dict


def create_retriever_and_description_dicts(params_dict, filestore, doc_dict=doc_dict, vector_dict=vector_dict):
    retriever_dict = dict()
    description_dict = dict()
    for doc_id in doc_dict:
        retriever_dict[doc_id] = create_retriever(
            doc_dict[doc_id], params_dict[doc_id]['site_key'], 
            filestore,
            vector_dict=vector_dict, 
            text_splitter=params_dict[doc_id].get('text_splitter', None)
            )
        description_dict[params_dict[doc_id]['site_key']] = params_dict[doc_id]['doc_description']

    return retriever_dict, description_dict

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list


recylebc = """
This document provides information from the Recycle BC website or BC government 
website. It has the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""

CoV_mattress = """
Information from the City of Vancouver website about how to recycle mattresses.
"""

params_dict = {
    # 1: {
    #     'site_key': 'recycle',
    #     'doc_description': recylebc,
    #     'text_splitter': None
    # },
    2: {
        'site_key': 'mattress',
        'doc_description': CoV_mattress,
        'text_splitter': None
    }
}
filestore = '../embeddings/'

doc_id = 2
try:
    directory = 'data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)
except:
    directory = '../data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)


retriever_dict, description_dict = create_retriever_and_description_dicts(params_dict, filestore)

100%|██████████| 1/1 [00:00<00:00, 497.43it/s]

Number of files: 1


In [30]:
retriever_dict[2]['mattress']

In [46]:
vars(retriever_dict[2]['mattress']).keys()

dict_keys(['document_embedding_store', 'underlying_embeddings'])

In [42]:
getattr(retriever_dict[2]['mattress'], 'model')

AttributeError: 'CacheBackedEmbeddings' object has no attribute 'model'

In [44]:
getattr(retriever_dict[2]['mattress'], 'document_embedding_store')

In [49]:
type(getattr(retriever_dict[2]['mattress'], 'document_embedding_store'))

langchain.storage.encoder_backed.EncoderBackedStore

In [50]:
retriever_dict[2]

{'mattress': <langchain.embeddings.cache.CacheBackedEmbeddings at 0x250d67353d0>}

In [33]:
site_key = 'mattress'
vector_dict[site_key] = FAISS.from_documents(doc_dict[doc_id], retriever_dict[2][site_key])

AttributeError: 'str' object has no attribute 'mget'

## 2.21 Successfully saved embedding

In [2]:
import os

# documents
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.storage import LocalFileStore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

# from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool

# Creating the Agent
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

# Create memory 
from langchain.memory import ConversationBufferMemory
from langchain.memory import StreamlitChatMessageHistory # for Streamlit

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.agents import AgentExecutor
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

import streamlit as st


def create_documents(directory='../data', glob='**/[!.]*', show_progress=True, loader_cls=CSVLoader):
    loader = DirectoryLoader(
        directory, glob=glob, show_progress=show_progress,
        loader_cls=loader_cls)

    documents = loader.load()
    print(f'Number of files: {len(documents)}')
    return documents
    
def create_documents_from_csv(file_path='../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv'):
    loader = CSVLoader(file_path, encoding='utf-8')
    documents = loader.load()
    return documents

def create_retriever(
    documents, site_key, filestore, 
    embeddings_dict=embeddings_dict, 
    vector_dict=vector_dict, text_splitter=None
    ):
    """
    Parameters:
        - text_splitter (optional): a text splitter object. If None, the documents are not split. 
    """
    if text_splitter is None: # object type is the same (class 'langchain.schema.document.Document') whether or not the documents are split
        texts = documents
    else:
        texts = text_splitter.split_documents(documents)
   
    underlying_embeddings = OpenAIEmbeddings(
        openai_organization=os.environ['openai_organization'],
        openai_api_key=os.environ['openai_api_key']
        )
    embeddings_dict[site_key] = CacheBackedEmbeddings.from_bytes_store(
        underlying_embeddings, filestore, 
        namespace=f'{site_key}_{underlying_embeddings.model}'
        )
    vector_dict[site_key] = FAISS.from_documents(texts, embeddings_dict[site_key])
    retriever_dict[site_key] = vector_dict[site_key].as_retriever()
    return retriever_dict
    # return embeddings_dict


def create_retriever_and_description_dicts(params_dict, filepath, doc_dict=doc_dict, vector_dict=vector_dict):
    retriever_dict = dict()
    description_dict = dict()
    for doc_id in doc_dict:
        retriever_dict[doc_id] = create_retriever(
            doc_dict[doc_id], params_dict[doc_id]['site_key'], 
            LocalFileStore(filepath),
            vector_dict=vector_dict, 
            text_splitter=params_dict[doc_id].get('text_splitter', None)
            )
        description_dict[params_dict[doc_id]['site_key']] = params_dict[doc_id]['doc_description']

    return retriever_dict, description_dict

def create_tools_list(retriever_dict, description_dict):
    """
    https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html?highlight=create_retriever_tool#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool
    """
    tools_list = []
    for site_key, retriever in retriever_dict.items():
        tool_name = f'search_{site_key}'
        tool = create_retriever_tool(retriever_dict[site_key], tool_name, description_dict[site_key])
        tools_list.append(tool)
    return tools_list


recylebc = """
This document provides information from the Recycle BC website or BC government 
website. It has the most specific information 
about whether or not an item is accepted for recycling and where to recycle it.
This should be the main resource for recycling information for residents of British Columbia.
"""

CoV_mattress = """
Information from the City of Vancouver website about how to recycle mattresses.
"""

params_dict = {
    # 1: {
    #     'site_key': 'recycle',
    #     'doc_description': recylebc,
    #     'text_splitter': None
    # },
    2: {
        'site_key': 'mattress',
        'doc_description': CoV_mattress,
        'text_splitter': None
    }
}
filestore = '../embeddings/'
# doc_id = 1
# try:
#     directory = 'data'
#     doc_dict[doc_id] = create_documents(directory=directory, glob='*.csv')
# except:
#     doc_dict[doc_id] = create_documents_from_csv()
#     print('Done creating doc from CSV')

doc_id = 2
try:
    directory = 'data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)
except:
    directory = '../data'
    doc_dict[doc_id] = create_documents(directory=directory, glob='*.txt', loader_cls=TextLoader)



retriever_dict, description_dict = create_retriever_and_description_dicts(params_dict, filestore)

100%|██████████| 1/1 [00:00<00:00, 116.90it/s]


Number of files: 1


In [3]:
embeddings_dict

{'mattress': <langchain.embeddings.cache.CacheBackedEmbeddings at 0x20fe9689150>}

In [6]:
filepath = '../embeddings/'
list(LocalFileStore(filepath).yield_keys())

['mattress_text-embedding-ada-0024d698914-67ab-531b-a43a-655989f5061e']

### Repeat with Recycle BC doc

In [17]:
params_dict = {
    1: {
        'site_key': 'recycle',
        'doc_description': recylebc,
        'text_splitter': None
    },
    # 2: {
    #     'site_key': 'mattress',
    #     'doc_description': CoV_mattress,
    #     'text_splitter': None
    # }
}
filestore = '../embeddings/'
doc_id = 1
try:
    directory = 'data'
    doc_dict2[doc_id] = create_documents(directory=directory, glob='*.csv')
except:
    doc_dict2[doc_id] = create_documents_from_csv()
    print('Done creating doc from CSV')

retriever_dict, description_dict = create_retriever_and_description_dicts(params_dict, filestore, doc_dict=doc_dict2)

Done creating doc from CSV


In [18]:
doc_dict2

{1: [Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Newspapers, inserts and flyers\nSource: https://recyclebc.ca/what-can-i-recycle-2/', metadata={'source': '../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv', 'row': 0}),
  Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Magazines and catalogues\nSource: https://recyclebc.ca/what-can-i-recycle-2/', metadata={'source': '../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv', 'row': 1}),
  Document(page_content='Conclusion: Accepted\nLocation: Curbside collection, Multi-family collection, Recycle BC depots\nCategory: Paper\nItem: Telephone books and other directories\nSource: https://recyclebc.ca/what-can-i-recycle-2/', metadata={'source': '../data/Datajam_2023___Fine_Tuning_ChatBot_CSV_-_Recycle_BC_1.csv', 'row': 2}),
  D

In [19]:
retriever_dict

{1: {'recycle': VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x0000020FFEFC02D0>, search_type='similarity', search_kwargs={})}}

In [20]:
description_dict

{'recycle': '\nThis document provides information from the Recycle BC website or BC government \nwebsite. It has the most specific information \nabout whether or not an item is accepted for recycling and where to recycle it.\nThis should be the main resource for recycling information for residents of British Columbia.\n'}

In [21]:
embeddings_dict

{'mattress': <langchain.embeddings.cache.CacheBackedEmbeddings at 0x20fff009610>,
 'recycle': <langchain.embeddings.cache.CacheBackedEmbeddings at 0x20fe245d510>}

# *End of Page*